In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
events = {}
evt: LogResult
for k, evt in enumerate(ReadEvents().apply(web3,
                                            start_block=start,
                                            end_block=end,
                                            filter=filter,
                            )):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,279,901 tx:0xd0865e7c431c098e30b9b2362aa4c70ccf344d6357c723b0c6d160657c117deb
Swap at block:60,279,901 tx:0x810f4b4afe782fa269673a3b643e727b024302238681f018c44640f8897147da
Swap at block:60,279,901 tx:0xdddb35b0f869c2bca05c6b78cbbf635fb942e59b6845979995742991043624ec
Swap at block:60,279,901 tx:0xbffab46b67dd1b5b7540abf5303ab8159918c1c742e1ffa8a7f3440f6fde269f
Swap at block:60,279,901 tx:0xbffab46b67dd1b5b7540abf5303ab8159918c1c742e1ffa8a7f3440f6fde269f
Swap at block:60,279,905 tx:0x05ea37305ff8c1d17f76ea5febed84836c3124e2003a65a4dd642021438d3f36
Swap at block:60,279,907 tx:0x874b5f206bb47340be3cbb67893bd4fd30167423f4385d2a2a5eaabed5cbce04
Swap at block:60,279,909 tx:0x7707b5d3ec6968217b1697860d0e73a567cf0871fc6ad2984c329be53b4fc274
Swap at block:60,279,909 tx:0xe4009795437d6214875ad55ff341349caa6c7f349759aec75b49a3ada2c7fa06
Swap at block:60,279,909 tx:0xe4009795437d6214875ad55ff341349caa6c7f349759aec75b49a3ada2c7fa06
Swap at block:60,279,909 tx:0x2b167e715d21a10c48f6

In [7]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0xe8654f2b0a038a01bc273a2a7b7c48a76c0e58c5,0xd0865e7c431c098e30b9b2362aa4c70ccf344d6357c7...,60279901,1722987702,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0x486305621f07f5b3c598a48c31aaf3756d7095d5,0x810f4b4afe782fa269673a3b643e727b024302238681...,60279901,1722987702,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0x6c3849d4ba7fddc228147c92adbebaceaa456d3a,0xdddb35b0f869c2bca05c6b78cbbf635fb942e59b6845...,60279901,1722987702,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0x6e7a5fafcec6bb1e78bae2a1f0b612012bf14827,0xbffab46b67dd1b5b7540abf5303ab8159918c1c742e1...,60279901,1722987702,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0x6c3849d4ba7fddc228147c92adbebaceaa456d3a,0xbffab46b67dd1b5b7540abf5303ab8159918c1c742e1...,60279901,1722987702,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0x100e0cfcadb56c05f614502b4fe8e4f7a501ef34,0x05ea37305ff8c1d17f76ea5febed84836c3124e2003a...,60279905,1722987712,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0x874b5f206bb47340be3cbb67893bd4fd30167423f438...,60279907,1722987716,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0xa0a2e0238b3198e83c7dc6575cbbeb45ae83b4d9,0x7707b5d3ec6968217b1697860d0e73a567cf0871fc6a...,60279909,1722987720,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv2pair,swap,sushi,0x1587663e8f475e69ea2dbb38482c8c4ee9f388fb,0xe4009795437d6214875ad55ff341349caa6c7f349759...,60279909,1722987720,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv2pair,swap,sushi,0xb1f3555a7c3753ab4e6df1d66cfdb25477a36ce7,0xe4009795437d6214875ad55ff341349caa6c7f349759...,60279909,1722987720,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [8]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xe8654f2b0a038a01bc273a2a7b7c48a76c0e58c5',
  'tx_hash': '0xd0865e7c431c098e30b9b2362aa4c70ccf344d6357c723b0c6d160657c117deb',
  'blk_num': 60279901,
  'timestamp': 1722987702,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xC9E1aca5bd59470b5f05738867Baab35a235158d',
   'token1': '0xC9E1aca5bd59470b5f05738867Baab35a235158d',
   'amount0In': 0,
   'amount1Out': 0}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x486305621f07f5b3c598a48c31aaf3756d7095d5',
  'tx_hash': '0x810f4b4afe782fa269673a3b643e727b024302238681f018c44640f8897147da',
  'blk_num': 60279901,
  'timestamp': 1722987702,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xeE64D5765A3B95d55630410ec3B3EbBeAc46c862',
   'token1': '0xeE64D5765A3B95d55630410ec3B3EbBeAc46c862',
   'amount0In': 0,
   'amount1Out':